In [5]:
import os
from PIL import Image
import numpy as np
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import mean_squared_error as mse

# Process Images

In this notebook, we will be creating two sets of images: Low-Resolution (LR) and High-Resolution (HR) images. The LR images will be created by downscaling the HR images.

The workflow of creating training and testing data is as follows:
1. Load the HR images.
2. Resize every image to make all the images of the same size.
3. Downscale the images to create LR images.
4. Upscale the LR images to create LR images of the same size as HR images.
5. Save the HR and LR images in separate directories.

In [2]:
# File Paths
input_dir = 'Dataset/DIV2K_train_HR/'
resize_dir = 'Dataset/Resized/'
output_dir = 'Dataset/Processed/'
crop_dir = 'Dataset/Cropped/'

# Create output directories
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
if not os.path.exists(resize_dir):
    os.makedirs(resize_dir)
if not os.path.exists(crop_dir):
    os.makedirs(crop_dir)
    
# # Resize Size
# resize_size = (256, 256)
# 
# # Resize Images
# for filename in os.listdir(input_dir):
#     if filename.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
#         # Open the image
#         img_path = os.path.join(input_dir, filename)
#         img = Image.open(img_path)
# 
#         # Resize the image
#         img = img.resize(resize_size, Image.BICUBIC)
# 
#         # Save the image
#         output_path = os.path.join(resize_dir, filename)
#         img.save(output_path)

In [3]:
x1, y1 = 0, 0  # Top left corner of the crop box
crop_box = (x1, y1, x1 + 512, y1 + 512)  # Adjust these coordinates

# Crop Images
for filename in os.listdir(input_dir):
    if filename.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
        # Open the image
        img_path = os.path.join(input_dir, filename)
        img = Image.open(img_path)

        # Crop the image
        cropped_img = img.crop(crop_box)

        # Save the cropped image
        output_path = os.path.join(crop_dir, filename)
        cropped_img.save(output_path)

In [11]:
# Scale Factor    
def downscale_images(images, scaling_factor):
    # Upscale Images
    for filename in os.listdir(images):
        if filename.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            # Open the image
            img_path = os.path.join(images, filename)
            img = Image.open(img_path)
    
            # Calculate the new dimensions
            new_width = img.width // scaling_factor
            new_height = img.height // scaling_factor
    
            # Downscale the image
            img_downscaled = img.resize((new_width, new_height), Image.BICUBIC)
    
            # Upscale the image
            img_upscaled = img_downscaled.resize((img.width, img.height), Image.BICUBIC)
    
            # Save the downscaled image
            output_path = os.path.join(output_dir, filename)
            img_upscaled.save(output_path)
            
# Downscale Images
downscale_images(crop_dir, 3)

# Creating Metrics

In order to tell the difference between the HR and LR images, we will be using the following metrics:
1. Peak Signal-to-Noise Ratio (PSNR)
2. Structural Similarity Index (SSIM)
3. Mean Squared Error (MSE)

These metrics will help us evaluate the performance of the model.

In [6]:
# Calculate Metrics
def calculate_metrics(hr_img, lr_img):
    # Convert images to numpy arrays
    hr_data = np.array(hr_img).astype(np.float32)
    lr_data = np.array(lr_img).astype(np.float32)
    # Calculate PSNR
    psnr_val = psnr(hr_data, lr_data, data_range=255)
    # Calculate SSIM
    ssim_val = ssim(hr_data, lr_data, multichannel=True, channel_axis=2, data_range=255)
    # Calculate MSE
    mse_val = mse(hr_data, lr_data)
    return psnr_val, ssim_val, mse_val

In [7]:
# Dimension of images
hr_img = Image.open('Dataset/Cropped/0001.png')
lr_img = Image.open('Dataset/Processed/0001.png')
hr_img.size, lr_img.size

((512, 512), (512, 512))

In [8]:
psnr_results, ssim_results, mse_results = [], [], []
test_dir = 'Dataset/Cropped/'
output_dir = 'Dataset/Processed/'
for hr, lr in zip(os.listdir(test_dir), os.listdir(output_dir)):
    # Open the images
    hr_img = Image.open(os.path.join(test_dir, hr))
    lr_img = Image.open(os.path.join(output_dir, lr))

    # Calculate the metrics
    psnr_val, ssim_val, mse_val = calculate_metrics(hr_img, lr_img)

    # Append the metrics to the new variables
    psnr_results.append(psnr_val)
    ssim_results.append(ssim_val)
    mse_results.append(mse_val)

# Calculate the average metrics
psnr_avg = np.mean(psnr_results)
ssim_avg = np.mean(ssim_results)
mse_avg = np.mean(mse_results)

# Print the results
print(f'Average PSNR: {psnr_avg:.4f} dB')
print(f'Average SSIM: {ssim_avg:.4f}')
print(f'Average MSE: {mse_avg:.4f}')

Average PSNR: 33.8643 dB
Average SSIM: 0.8837
Average MSE: 98.2546
